In [1]:
import pandas as pd
from scipy import stats
import matplotlib.pyplot as plt
import os.path
import numpy as np
import math

# Projeto 2 - Classificador Automático de Sentimento

Você foi contratado por uma empresa parar analisar como os clientes estão reagindo a um determinado produto no Twitter. A empresa deseja que você crie um programa que irá analisar as mensagens disponíveis e classificará como "relevante" ou "irrelevante". Com isso ela deseja que mensagens negativas, que denigrem o nome do produto, ou que mereçam destaque, disparem um foco de atenção da área de marketing.<br /><br />
Como aluno de Ciência dos Dados, você lembrou do Teorema de Bayes, mais especificamente do Classificador Naive-Bayes, que é largamente utilizado em filtros anti-spam de e-mails. O classificador permite calcular qual a probabilidade de uma mensagem ser relevante dadas as palavras em seu conteúdo.<br /><br />
Para realizar o MVP (*minimum viable product*) do projeto, você precisa implementar uma versão do classificador que "aprende" o que é relevante com uma base de treinamento e compara a performance dos resultados com uma base de testes.<br /><br />
Após validado, o seu protótipo poderá também capturar e classificar automaticamente as mensagens da plataforma.

## Informações do Projeto

Prazo: 19/Set até às 23:59.<br />
Grupo: 2 ou 3 pessoas - grupos com 3 pessoas terá uma rubrica diferenciada.<br /><br />
Entregáveis via GitHub: 
* Arquivo notebook com o código do classificador, seguindo as orientações abaixo.
* Arquivo Excel com as bases de treinamento e teste totalmente classificado.

**NÃO gravar a key do professor no arquivo**


### Entrega Intermediária: Check 1 - APS 2

Até o dia 10/Set às 23:59, xlsx deve estar no Github com as seguintes evidências: 

  * Produto escolhido.
  * Arquivo Excel contendo a base de treinamento e a base de testes já classificadas.

Sugestão de leitura:<br />
https://monkeylearn.com/blog/practical-explanation-naive-bayes-classifier/

___

## Parte I - Adquirindo a Base de Dados

Acessar o notebook **Projeto-2-Planilha** para realizar a coleta dos dados. O grupo deve classificar os dados coletados manualmente.

___
## Parte II - Montando o Classificador Naive-Bayes

Com a base de treinamento montada, comece a desenvolver o classificador. Não se esqueça de implementar o Laplace Smoothing (https://en.wikipedia.org/wiki/Laplace_smoothing).

Opcionalmente: 
* Limpar as mensagens removendo os caracteres: enter, :, ", ', (, ), etc. Não remover emojis.<br />
* Corrigir separação de espaços entre palavras e/ou emojis.
* Propor outras limpezas/transformações que não afetem a qualidade da informação.

Escreva o seu código abaixo:

In [2]:
tweets = pd.read_excel("tweets_spider man_201809191428.xlsx") #lê o excel
lista_lixo = [',','@','/',':',';','r$','\n','\(','\)','#','\?','-','"','\[','\]'] #lista com o que queremos remover dos tweets

for e in lista_lixo: #Remove o 'Lixo'
    tweets.Treinamento = tweets.Treinamento.replace(e,'',regex=True)
tweets.head(10)

Treinamento  Relevancia
0  gostei de um vídeo youtube httpst.co6ubtbkqzpi...           1
1  o vídeo do o miranha jogando spider man ta mui...           1
2  gostei de um vídeo youtube httpst.coweckqzivxw...           1
3  papel de parede gratuito de filmes homemaranha...           0
4  rt jojo16644573 ei_games17 spidermannoeigames ...           1
5  rt lukas_djokovic spidermannoeigames muito leg...           1
6  rt promobitoficial promoção oferta  cartão ame...           0
7  rt ignbrasil estúdio fala sobre traje simbiont...           1
8  i liked a youtube video httpst.co7ghxahowsi o ...           1
9  gostei de um vídeo youtube httpst.coasfotncy0v...           1

In [3]:
strain_col = pd.Series(np.sum(tweets.Treinamento).split()) #Separa as palavras em strings
relevs_col = tweets.Relevancia #Cria uma coluna com as relevâncias 
strain_col.value_counts().head(10) #Imprime e conta palavras iguais

spider     269
youtube    131
de         130
man        127
jogando    122
um         120
miranha    112
o          106
vídeo       99
a           75
dtype: int64

In [4]:
prob = relevs_col.value_counts()
prob

1    229
0     71
Name: Relevancia, dtype: int64

In [5]:
prob_relevante = prob[1]/len(relevs_col) #Calcula a provabilidade de uma palavra ser relevante entre todas as palavras do DataFrame
prob_irrelevante = prob[0]/len(relevs_col) #Calcula a provabilidade de uma palavra ser irrelevante entre todas as palavras do DataFrame
print(int(prob_relevante * 100),'%', int(prob_irrelevante * 100),'%')


76 % 23 %


In [6]:
tweets_relevantes = tweets[tweets.Relevancia == 1] #Separa os tweets relevantes
relevs = np.sum(tweets_relevantes.Treinamento).split()
Quantidade_em_relev =  pd.Series(relevs)
all_relevs = len(relevs)
Quantidade_em_relev.value_counts().head(20) #Imprime uma coluna com as quantidades de cada plavra dentro do grupo de relevantes

spider                212
youtube               122
jogando               120
miranha               112
um                    108
de                    103
vídeo                  91
man                    82
o                      76
a                      61
mangostei              46
jogo                   45
do                     41
video                  32
liked                  32
e                      29
que                    26
uma                    25
spidermannoeigames     21
man'                   19
dtype: int64

In [7]:
tweets_irrelevantes = tweets[tweets.Relevancia == 0] #Separa os tweets irrelevantes
irrelevs = np.sum(tweets_irrelevantes.Treinamento).split()
Quantidade_em_irrelev =  pd.Series(irrelevs)
all_irrelevs = len(irrelevs)
Quantidade_em_irrelev.value_counts().head(20) #Imprime uma coluna com as quantidades de cada plavra dentro do grupo de relevantes

spider     57
man        45
o          30
e          30
de         27
que        22
do         16
eu         15
a          14
um         12
uma        12
se         10
não        10
youtube     9
jogo        9
amo         8
vídeo       8
ps4         7
da          7
no          7
dtype: int64

In [8]:
def Naive(Tweet):
    naive = Tweet.split(' ')
    rele = []
    irrele = []
    for i in range(len(naive)):
        rele.append((relevs.count(naive[i]) + 1) / (all_relevs + len(naive)))
        irrele.append((irrelevs.count(naive[i]) + 1) / (all_irrelevs + len(naive)))
    prop_rel = np.prod(rele)
    prop_irrel = np.prod(irrele)
    if prop_rel > prop_irrel:
        return 1
    else:
        return 0


___
## Verificando a performance

Agora você deve testar o seu Classificador com a base de Testes.<br /><br /> 

Você deve extrair as seguintes medidas:
* Porcentagem de positivos falsos (marcados como relevante mas não são relevantes)
* Porcentagem de positivos verdadeiros (marcado como relevante e são relevantes)
* Porcentagem de negativos verdadeiros (marcado como não relevante e não são relevantes)
* Porcentagem de negativos falsos (marcado como não relevante e são relevantes)

Obrigatório para grupos de 3 alunos:
* Criar categorias intermediárias de relevância baseado na diferença de probabilidades. Exemplo: muito relevante, relevante, neutro, irrelevante e muito irrelevante.

In [9]:
lista_performance = []
for e in tweets.Treinamento:
    lista_performance.append(Naive(e))
    
tweets["Treino"] = lista_performance

tweets

Treinamento  Relevancia  Treino
0    gostei de um vídeo youtube httpst.co6ubtbkqzpi...           1       1
1    o vídeo do o miranha jogando spider man ta mui...           1       1
2    gostei de um vídeo youtube httpst.coweckqzivxw...           1       1
3    papel de parede gratuito de filmes homemaranha...           0       0
4    rt jojo16644573 ei_games17 spidermannoeigames ...           1       1
5    rt lukas_djokovic spidermannoeigames muito leg...           1       1
6    rt promobitoficial promoção oferta  cartão ame...           0       0
7    rt ignbrasil estúdio fala sobre traje simbiont...           1       0
8    i liked a youtube video httpst.co7ghxahowsi o ...           1       1
9    gostei de um vídeo youtube httpst.coasfotncy0v...           1       1
10   i liked a youtube video httpst.co05rm9fnkls o ...           1       1
11   rt spideyquillz fordowneyjr psychounicorn_ e a...           0       0
12   marvel's spider man 7 missão camada sobre cama...           1       0
13   rt zaxin_ eu não sei porq amo tanto essas cois...           0       0
14          o ric so mete fatos de merda no spider man           0       0
15   i liked a youtube video httpst.coa0uvnlu0zi o ...           1       1
16   rt unlockscreen ☆spider man•rt se salvoufav se...           0       0
17   gostei de um vídeo youtube httpst.cosqb2achcix...           1       1
18   gostei de um vídeo youtube httpst.cozpy8wawzjc...           1       1
19   gostei de um vídeo youtube httpst.cotjgnxzz8qx...           1       1
20   o miranha jogando spider man httpst.co8npo6vac...           1       1
21   gostei de um vídeo youtube httpst.co8cqc50sa4f...           0       0
22   i liked a youtube video httpst.coasxsicxmms o ...           1       1
23   rt lukas_djokovic spidermannoeigames vou ter m...           1       1
24   gostei de um vídeo youtube httpst.coc1pqfcdzsz...           1       1
25   gostei de um vídeo youtube httpst.copkoxkllnm7...           1       1
26   e eu q premeditei o nome do drone do spider ma...           0       0
27   adicionei um vídeo a uma playlist youtube http...           1       1
28   gostei de um vídeo youtube httpst.como3cstwb3e...           1       1
29   um dia normal na minha vida e jogar spider man...           1       0
..                                                 ...         ...     ...
270  i liked a youtube video httpst.cotb2yxeifzo o ...           1       1
271  spider man ps4 jogo ou filmeesquiva por baixo ...           1       1
272  i liked a youtube video httpst.coxzufzioiuk o ...           1       1
273  gostei de um vídeo youtube httpst.copjaxhe5l7n...           1       1
274               pqp spider man ta mt lindo esse game           1       0
275  acabei o modo história do spider man 😭😭😭 ca pu...           1       0
276  rt gleicedias senhor barriga ligando pro spide...           0       0
277  rt theenemybr insomniac sugere anúncio de novo...           0       0
278  gostei de um vídeo youtube httpst.co9j5r3zio86...           1       1
279  gostei de um vídeo youtube httpst.cohua50otbdh...           1       1
280  gostei de um vídeo youtube httpst.coqniwsvoct6...           1       1
281  teatrobonecos e tá acontecendo com o homem ara...           1       0
282  agora não preciso dormir pq tenho spider man h...           1       0
283  gostei de um vídeo youtube httpst.corldrjxnj1a...           1       1
284  gostei de um vídeo youtube httpst.co1lyly77l3r...           1       1
285  gostei de um vídeo youtube httpst.copa5texk4yv...           1       1
286  spider man do play4 seria o jogo q nao ia faze...           1       0
287  rt unlockscreen ☆spider man•rt se salvoufav se...           0       0
288  micalibruno to nessa ressaca tbm. aluguei farc...           1       0
289  i liked a youtube video httpst.constjfksa8g o ...           1       1
290           o miranha jogando spider man é muito bom           1       1
291  spider man  o início  01 httpst.cowrhlh2jxpl v...           1  

In [10]:
positivos_vdd = 0
positivos_mnt = 0

negativos_vdd = 0
negativos_mnt = 0


for e in range(len(tweets.Treino)):
    if tweets.Treino[e] == 1:
        if tweets.Relevancia[e] == tweets.Treino[e]:
            positivos_vdd += 1
        else:
            positivos_mnt += 1
    else:
        if tweets.Relevancia[e] == tweets.Treino[e]:
            negativos_vdd += 1
        else:
            negativos_mnt +=1

print('Porcentagem de positivos verdadeiros: {0}%'.format(round((positivos_vdd/(positivos_vdd+positivos_mnt))*100, 2)))
print('Porcentagem de positivos falsos: {0}%'.format(round((positivos_mnt/(positivos_vdd+positivos_mnt))*100, 2)))
print('Porcentagem de negativos verdadeiros: {0}%'.format(round((negativos_vdd/(negativos_vdd+negativos_mnt))*100, 2)))
print('Porcentagem de negativos falsos: {0}%'.format(round((negativos_mnt/(negativos_vdd+negativos_mnt))*100, 2)))

Porcentagem de positivos verdadeiros: 99.42%
Porcentagem de positivos falsos: 0.58%
Porcentagem de negativos verdadeiros: 54.26%
Porcentagem de negativos falsos: 45.74%


___
## Concluindo

Escreva aqui a sua conclusão.<br /> 
Faça um comparativo qualitativo sobre as medidas obtidas.<br />
Explique como são tratadas as mensagens com dupla negação e sarcasmo.<br />
Proponha um plano de expansão. Por que eles devem continuar financiando o seu projeto?<br />

Opcionalmente: 
* Discorrer por que não posso alimentar minha base de Treinamento automaticamente usando o próprio classificador, aplicado a novos tweets.
* Propor diferentes cenários de uso para o classificador Naive-Bayes. Cenários sem intersecção com este projeto.
* Sugerir e explicar melhorias reais no classificador com indicações concretas de como implementar (não é preciso codificar, mas indicar como fazer e material de pesquisa sobre o assunto).
